# 接ARIMA_Purchase_Redeem
从ARIMA的结果上看，并不能把每周的这种周期性规律表现出来（周末低、周初高）
故现在用rule base的方式进行数据挖掘，将周期性的规律展现出来

In [1]:
import pandas as pd

#数据加载
data = pd.read_csv('./user_balance_table.csv', parse_dates=['report_date'])
data

user_id report_date  tBalance  yBalance  total_purchase_amt  \
0              1  2014-08-05     20385     20383                   2   
1              1  2014-08-08     20391     20389                   2   
2              1  2014-08-11     20397     20395                   2   
3              1  2014-08-14     20403     20401                   2   
4              1  2014-08-17     20409     20407                   2   
...          ...         ...       ...       ...                 ...   
2840416    28033  2014-08-25    550646    550585                  61   
2840417    28033  2014-08-31    525707    538147                  60   
2840418    28033  2014-07-24  20487121  20484824                2297   
2840419    28033  2014-07-27  20462288  20491722                2298   
2840420    28035  2014-03-05         0         0                   0   

         direct_purchase_amt  purchase_bal_amt  purchase_bank_amt  \
0                          0                 0                  0   
1                          0                 0                  0   
2                          0                 0                  0   
3                          0                 0                  0   
4                          0                 0                  0   
...                      ...               ...                ...   
2840416                    0                 0                  0   
2840417                    0                 0                  0   
2840418                    0                 0                  0   
2840419                    0                 0                  0   
2840420                    0                 0                  0   

         total_redeem_amt  consume_amt  transfer_amt  tftobal_amt  \
0                       0            0             0            0   
1                       0            0             0            0   
2                       0            0             0            0   
3                       0            0             0            0   
4                       0            0             0            0   
...                   ...          ...           ...          ...   
2840416                 0            0             0            0   
2840417             12500        12500             0            0   
2840418                 0            0             0            0   
2840419             31732            0         31732            0   
2840420                 0            0             0            0   

         tftocard_amt  share_amt  category1  category2  category3  category4  
0                   0          2        NaN        NaN        NaN        NaN  
1                   0          2        NaN        NaN        NaN        NaN  
2                   0          2        NaN        NaN        NaN        NaN  
3                   0          2        NaN        NaN        NaN        NaN  
4                   0          2        NaN        NaN        NaN        NaN  
...               ...        ...        ...        ...        ...        ...  
2840416             0         61        NaN        NaN        NaN        NaN  
2840417             0         60        0.0        0.0        0.0    12500.0  
2840418             0       2297        NaN        NaN        NaN        NaN  
2840419         31732       2298        NaN        NaN        NaN        NaN  
2840420             0          0        NaN        NaN        NaN        NaN  

[2840421 rows x 18 columns]

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2840421 entries, 0 to 2840420
Data columns (total 18 columns):
 #   Column               Dtype         
---  ------               -----         
 0   user_id              int64         
 1   report_date          datetime64[ns]
 2   tBalance             int64         
 3   yBalance             int64         
 4   total_purchase_amt   int64         
 5   direct_purchase_amt  int64         
 6   purchase_bal_amt     int64         
 7   purchase_bank_amt    int64         
 8   total_redeem_amt     int64         
 9   consume_amt          int64         
 10  transfer_amt         int64         
 11  tftobal_amt          int64         
 12  tftocard_amt         int64         
 13  share_amt            int64         
 14  category1            float64       
 15  category2            float64       
 16  category3            float64       
 17  category4            float64       
dtypes: datetime64[ns](1), float64(4), int64(13)
memory usage: 390.

In [3]:
# 给数据添加时间维度
def add_timestamp(data):
    # 时间格式转换
    data['report_date'] = pd.to_datetime(data['report_date'],format='%Y%m%d')
    # 添加时间维度
    data['day'] = data['report_date'].dt.day
    data['month'] = data['report_date'].dt.month
    data['year'] = data['report_date'].dt.year
    data['week'] = data['report_date'].dt.week #在一年中的第多少个week
    data['weekday'] = data['report_date'].dt.weekday
    return data

data = add_timestamp(data)
data

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':


user_id report_date  tBalance  yBalance  total_purchase_amt  \
0              1  2014-08-05     20385     20383                   2   
1              1  2014-08-08     20391     20389                   2   
2              1  2014-08-11     20397     20395                   2   
3              1  2014-08-14     20403     20401                   2   
4              1  2014-08-17     20409     20407                   2   
...          ...         ...       ...       ...                 ...   
2840416    28033  2014-08-25    550646    550585                  61   
2840417    28033  2014-08-31    525707    538147                  60   
2840418    28033  2014-07-24  20487121  20484824                2297   
2840419    28033  2014-07-27  20462288  20491722                2298   
2840420    28035  2014-03-05         0         0                   0   

         direct_purchase_amt  purchase_bal_amt  purchase_bank_amt  \
0                          0                 0                  0   
1                          0                 0                  0   
2                          0                 0                  0   
3                          0                 0                  0   
4                          0                 0                  0   
...                      ...               ...                ...   
2840416                    0                 0                  0   
2840417                    0                 0                  0   
2840418                    0                 0                  0   
2840419                    0                 0                  0   
2840420                    0                 0                  0   

         total_redeem_amt  consume_amt  ...  share_amt  category1  category2  \
0                       0            0  ...          2        NaN        NaN   
1                       0            0  ...          2        NaN        NaN   
2                       0            0  ...          2        NaN        NaN   
3                       0            0  ...          2        NaN        NaN   
4                       0            0  ...          2        NaN        NaN   
...                   ...          ...  ...        ...        ...        ...   
2840416                 0            0  ...         61        NaN        NaN   
2840417             12500        12500  ...         60        0.0        0.0   
2840418                 0            0  ...       2297        NaN        NaN   
2840419             31732            0  ...       2298        NaN        NaN   
2840420                 0            0  ...          0        NaN        NaN   

         category3  category4  day  month  year  week  weekday  
0              NaN        NaN    5      8  2014    32        1  
1              NaN        NaN    8      8  2014    32        4  
2              NaN        NaN   11      8  2014    33        0  
3              NaN        NaN   14      8  2014    33        3  
4              NaN        NaN   17      8  2014    33        6  
...            ...        ...  ...    ...   ...   ...      ...  
2840416        NaN        NaN   25      8  2014    35        0  
2840417        0.0    12500.0   31      8  2014    35        6  
2840418        NaN        NaN   24      7  2014    30        3  
2840419        NaN        NaN   27      7  2014    30        6  
2840420        NaN        NaN    5      3  2014    10        2  

[2840421 rows x 23 columns]

In [6]:
data['weekday'].value_counts() #0代表周一

6    410701
5    408824
4    407509
3    405907
2    404159
1    402536
0    400785
Name: weekday, dtype: int64

In [14]:
def get_total_balance(data,begin):
    # 在copy数据上进行后续操作
    df_temp = data.copy()
    # 按照date进行聚合
    df_temp = df_temp.groupby(['report_date'])['total_purchase_amt','total_redeem_amt'].sum()
    # 还原date字段，重新索引
    df_temp.reset_index(inplace=True)
    df_temp = df_temp[(df_temp['report_date'] >= begin)]
    return df_temp
    
# 筛选从2014-03-01之后的数据作为平稳数据
total_balance = get_total_balance(data,'2014-03-01')


C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


In [15]:
total_balance

report_date  total_purchase_amt  total_redeem_amt
243  2014-03-01           362865580         211279011
244  2014-03-02           276202230         246199417
245  2014-03-03           505305862         513017360
246  2014-03-04           524146340         250562978
247  2014-03-05           454295491         209072753
..          ...                 ...               ...
422  2014-08-27           302194801         468164147
423  2014-08-28           245082751         297893861
424  2014-08-29           267554713         273756380
425  2014-08-30           199708772         196374134
426  2014-08-31           275090213         292943033

[184 rows x 3 columns]

In [19]:
import datetime
import numpy as np

# 生成测试数据
def generate_test_data(data):
    # 在copy数据上进行操作
    total_balance = data.copy()
    # 生成2014-09-01到2014-09-30的数据
    start = datetime.datetime(2014,9,1)
    end = datetime.datetime(2014,10,1)
    testdata = []
    while start != end:
        # 添加三个字段：date，total_purchase_amt，total_redeem_amt
        temp = [start, np.nan, np.nan]
        testdata.append(temp)
        # 日期+1
        start += datetime.timedelta(days=1)
    # 封装testdata
    testdata = pd.DataFrame(testdata)
    testdata.columns = total_balance.columns
    # 将testdata合并到total_balance中
    total_balance = pd.concat([total_balance,testdata],axis=0)
    return total_balance

total_balance = generate_test_data(total_balance)
total_balance

report_date  total_purchase_amt  total_redeem_amt
243  2014-03-01         362865580.0       211279011.0
244  2014-03-02         276202230.0       246199417.0
245  2014-03-03         505305862.0       513017360.0
246  2014-03-04         524146340.0       250562978.0
247  2014-03-05         454295491.0       209072753.0
..          ...                 ...               ...
25   2014-09-26                 NaN               NaN
26   2014-09-27                 NaN               NaN
27   2014-09-28                 NaN               NaN
28   2014-09-29                 NaN               NaN
29   2014-09-30                 NaN               NaN

[214 rows x 3 columns]

In [20]:
# 对total_balance添加时间维度
total_balance = add_timestamp(total_balance)
total_balance

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':


report_date  total_purchase_amt  total_redeem_amt  day  month  year  week  \
243  2014-03-01         362865580.0       211279011.0    1      3  2014     9   
244  2014-03-02         276202230.0       246199417.0    2      3  2014     9   
245  2014-03-03         505305862.0       513017360.0    3      3  2014    10   
246  2014-03-04         524146340.0       250562978.0    4      3  2014    10   
247  2014-03-05         454295491.0       209072753.0    5      3  2014    10   
..          ...                 ...               ...  ...    ...   ...   ...   
25   2014-09-26                 NaN               NaN   26      9  2014    39   
26   2014-09-27                 NaN               NaN   27      9  2014    39   
27   2014-09-28                 NaN               NaN   28      9  2014    39   
28   2014-09-29                 NaN               NaN   29      9  2014    40   
29   2014-09-30                 NaN               NaN   30      9  2014    40   

     weekday  
243        5  
244        6  
245        0  
246        1  
247        2  
..       ...  
25         4  
26         5  
27         6  
28         0  
29         1  

[214 rows x 8 columns]

In [21]:
# 存储备份 容灾机制
temp = total_balance.copy()

In [106]:
# 就算total_balance发生了改变 也是改变的备份
total_balance = temp.copy()
total_balance

report_date  total_purchase_amt  total_redeem_amt  day  month  year  week  \
243  2014-03-01         362865580.0       211279011.0    1      3  2014     9   
244  2014-03-02         276202230.0       246199417.0    2      3  2014     9   
245  2014-03-03         505305862.0       513017360.0    3      3  2014    10   
246  2014-03-04         524146340.0       250562978.0    4      3  2014    10   
247  2014-03-05         454295491.0       209072753.0    5      3  2014    10   
..          ...                 ...               ...  ...    ...   ...   ...   
25   2014-09-26                 NaN               NaN   26      9  2014    39   
26   2014-09-27                 NaN               NaN   27      9  2014    39   
27   2014-09-28                 NaN               NaN   28      9  2014    39   
28   2014-09-29                 NaN               NaN   29      9  2014    40   
29   2014-09-30                 NaN               NaN   30      9  2014    40   

     weekday  
243        5  
244        6  
245        0  
246        1  
247        2  
..       ...  
25         4  
26         5  
27         6  
28         0  
29         1  

[214 rows x 8 columns]

In [107]:
# total_balance = total_balance[['report_date','total_purchase_amt','total_redeem_amt']]
# groupby('weekday',as_index=False) 可以将weekday与其它字段处于同一高度
weekday_weight = total_balance[['weekday','total_purchase_amt','total_redeem_amt']].groupby('weekday',as_index=False).mean()
weekday_weight


weekday  total_purchase_amt  total_redeem_amt
0        0        3.273693e+08      3.456808e+08
1        1        3.340518e+08      3.221266e+08
2        2        3.213877e+08      3.181960e+08
3        3        3.146910e+08      2.821480e+08
4        4        2.492893e+08      2.680463e+08
5        5        1.958666e+08      1.840544e+08
6        6        1.966477e+08      2.093138e+08

In [108]:
weekday_weight = weekday_weight.rename(columns={'total_purchase_amt':'purchase_weekday','total_redeem_amt':'redeem_weekday'})
weekday_weight

weekday  purchase_weekday  redeem_weekday
0        0      3.273693e+08    3.456808e+08
1        1      3.340518e+08    3.221266e+08
2        2      3.213877e+08    3.181960e+08
3        3      3.146910e+08    2.821480e+08
4        4      2.492893e+08    2.680463e+08
5        5      1.958666e+08    1.840544e+08
6        6      1.966477e+08    2.093138e+08

In [109]:
# 得到purchase、redeem的factor（因子）
weekday_weight['purchase_weekday'] /= np.mean(total_balance['total_purchase_amt'])
weekday_weight['redeem_weekday'] /= np.mean(total_balance['total_redeem_amt'])
weekday_weight

weekday  purchase_weekday  redeem_weekday
0        0          1.185411        1.257964
1        1          1.209608        1.172248
2        2          1.163752        1.157944
3        3          1.139503        1.026762
4        4          0.902682        0.975445
5        5          0.709237        0.669791
6        6          0.712065        0.761712

In [110]:
# 将total_balance与周期因子进行合并
total_balance = pd.merge(total_balance,weekday_weight,on='weekday',how='left')
total_balance

report_date  total_purchase_amt  total_redeem_amt  day  month  year  week  \
0    2014-03-01         362865580.0       211279011.0    1      3  2014     9   
1    2014-03-02         276202230.0       246199417.0    2      3  2014     9   
2    2014-03-03         505305862.0       513017360.0    3      3  2014    10   
3    2014-03-04         524146340.0       250562978.0    4      3  2014    10   
4    2014-03-05         454295491.0       209072753.0    5      3  2014    10   
..          ...                 ...               ...  ...    ...   ...   ...   
209  2014-09-26                 NaN               NaN   26      9  2014    39   
210  2014-09-27                 NaN               NaN   27      9  2014    39   
211  2014-09-28                 NaN               NaN   28      9  2014    39   
212  2014-09-29                 NaN               NaN   29      9  2014    40   
213  2014-09-30                 NaN               NaN   30      9  2014    40   

     weekday  purchase_weekday  redeem_weekday  
0          5          0.709237        0.669791  
1          6          0.712065        0.761712  
2          0          1.185411        1.257964  
3          1          1.209608        1.172248  
4          2          1.163752        1.157944  
..       ...               ...             ...  
209        4          0.902682        0.975445  
210        5          0.709237        0.669791  
211        6          0.712065        0.761712  
212        0          1.185411        1.257964  
213        1          1.209608        1.172248  

[214 rows x 10 columns]

In [111]:
# 计算日期因子
# 统计周一到周日，在1-31号中出现的频次['weekday','day']出现的频次
weekday_count = total_balance[['report_date','weekday','day']].groupby(['weekday','day'], as_index=False).count()
weekday_count = pd.merge(weekday_count,weekday_weight,on='weekday')
weekday_count

weekday  day  report_date  purchase_weekday  redeem_weekday
0          0    1            1          1.185411        1.257964
1          0    2            1          1.185411        1.257964
2          0    3            1          1.185411        1.257964
3          0    4            1          1.185411        1.257964
4          0    5            1          1.185411        1.257964
..       ...  ...          ...               ...             ...
179        6   27            2          0.712065        0.761712
180        6   28            1          0.712065        0.761712
181        6   29            1          0.712065        0.761712
182        6   30            1          0.712065        0.761712
183        6   31            1          0.712065        0.761712

[184 rows x 5 columns]

In [112]:
# 日期计算 = 周期因子 * （周一到周日在1-31号中出现的次数）/一共有几个月
weekday_count['purchase_weekday'] = weekday_count['purchase_weekday'] * weekday_count['report_date'] / len(np.unique(total_balance['month']))

In [113]:
weekday_count['redeem_weekday'] = weekday_count['redeem_weekday'] * weekday_count['report_date'] / len(np.unique(total_balance['month']))

In [114]:
# 得到日期因子
day_rate = weekday_count.drop(['weekday','report_date'],axis=1).groupby('day',as_index=False).sum()
day_rate

day  purchase_weekday  redeem_weekday
0     1          1.009731        1.005167
1     2          1.006644        1.021864
2     3          1.037011        1.010455
3     4          1.030815        1.046789
4     5          1.002776        0.989992
5     6          0.935559        0.932230
6     7          0.999723        1.015369
7     8          1.009731        1.005167
8     9          1.006644        1.021864
9    10          1.037011        1.010455
10   11          1.030815        1.046789
11   12          1.002776        0.989992
12   13          0.935559        0.932230
13   14          0.999723        1.015369
14   15          1.009731        1.005167
15   16          1.006644        1.021864
16   17          1.037011        1.010455
17   18          1.030815        1.046789
18   19          1.002776        0.989992
19   20          0.935559        0.932230
20   21          0.999723        1.015369
21   22          1.009731        1.005167
22   23          1.006644        1.021864
23   24          1.037011        1.010455
24   25          1.030815        1.046789
25   26          1.002776        0.989992
26   27          0.935559        0.932230
27   28          0.999723        1.015369
28   29          1.009731        1.005167
29   30          1.006644        1.021864
30   31          0.535174        0.530890

In [115]:
# 按照日期day计算均值，即1号的均值、2号的均值......
day_mean = total_balance[['day','total_purchase_amt','total_redeem_amt']].groupby('day',as_index=False).mean()
day_mean

day  total_purchase_amt  total_redeem_amt
0     1        3.253391e+08      2.347676e+08
1     2        2.481255e+08      2.186074e+08
2     3        2.994808e+08      2.968247e+08
3     4        3.159802e+08      2.925598e+08
4     5        3.286272e+08      2.552484e+08
5     6        2.998453e+08      2.460958e+08
6     7        2.836725e+08      2.374837e+08
7     8        2.918664e+08      2.475255e+08
8     9        2.856603e+08      2.440269e+08
9    10        3.446347e+08      2.593854e+08
10   11        2.864396e+08      3.051408e+08
11   12        2.747103e+08      2.548406e+08
12   13        2.479819e+08      2.740102e+08
13   14        2.708833e+08      2.950661e+08
14   15        2.958162e+08      2.302103e+08
15   16        3.144102e+08      3.124890e+08
16   17        2.565208e+08      2.987040e+08
17   18        2.803201e+08      2.798990e+08
18   19        2.836917e+08      2.608026e+08
19   20        2.911061e+08      2.367789e+08
20   21        2.815229e+08      2.817788e+08
21   22        2.430277e+08      2.612727e+08
22   23        2.282969e+08      2.625709e+08
23   24        2.407172e+08      2.973243e+08
24   25        2.430157e+08      3.156773e+08
25   26        2.503822e+08      3.116598e+08
26   27        2.383337e+08      3.178620e+08
27   28        2.662850e+08      3.460569e+08
28   29        2.443095e+08      2.691366e+08
29   30        2.393089e+08      2.937321e+08
30   31        2.531319e+08      2.842130e+08

In [116]:
# 去掉周期因素影响
day_pred = pd.merge(day_mean,day_rate,on='day',how='left')
day_pred['total_purchase_amt'] /= day_pred['purchase_weekday']
day_pred['total_redeem_amt'] /= day_pred['redeem_weekday']
day_pred

day  total_purchase_amt  total_redeem_amt  purchase_weekday  \
0     1        3.222038e+08      2.335608e+08          1.009731   
1     2        2.464879e+08      2.139300e+08          1.006644   
2     3        2.887923e+08      2.937537e+08          1.037011   
3     4        3.065345e+08      2.794832e+08          1.030815   
4     5        3.277176e+08      2.578287e+08          1.002776   
5     6        3.204986e+08      2.639860e+08          0.935559   
6     7        2.837511e+08      2.338892e+08          0.999723   
7     8        2.890537e+08      2.462531e+08          1.009731   
8     9        2.837750e+08      2.388057e+08          1.006644   
9    10        3.323346e+08      2.567017e+08          1.037011   
10   11        2.778770e+08      2.915018e+08          1.030815   
11   12        2.739499e+08      2.574168e+08          1.002776   
12   13        2.650629e+08      2.939297e+08          0.935559   
13   14        2.709584e+08      2.906000e+08          0.999723   
14   15        2.929654e+08      2.290269e+08          1.009731   
15   16        3.123351e+08      3.058029e+08          1.006644   
16   17        2.473655e+08      2.956135e+08          1.037011   
17   18        2.719404e+08      2.673883e+08          1.030815   
18   19        2.829065e+08      2.634391e+08          1.002776   
19   20        3.111575e+08      2.539918e+08          0.935559   
20   21        2.816010e+08      2.775138e+08          0.999723   
21   22        2.406857e+08      2.599297e+08          1.009731   
22   23        2.267902e+08      2.569529e+08          1.006644   
23   24        2.321259e+08      2.942481e+08          1.037011   
24   25        2.357512e+08      3.015674e+08          1.030815   
25   26        2.496891e+08      3.148104e+08          1.002776   
26   27        2.547501e+08      3.409693e+08          0.935559   
27   28        2.663588e+08      3.408189e+08          0.999723   
28   29        2.419552e+08      2.677531e+08          1.009731   
29   30        2.377294e+08      2.874474e+08          1.006644   
30   31        4.729902e+08      5.353523e+08          0.535174   

    redeem_weekday  
0         1.005167  
1         1.021864  
2         1.010455  
3         1.046789  
4         0.989992  
5         0.932230  
6         1.015369  
7         1.005167  
8         1.021864  
9         1.010455  
10        1.046789  
11        0.989992  
12        0.932230  
13        1.015369  
14        1.005167  
15        1.021864  
16        1.010455  
17        1.046789  
18        0.989992  
19        0.932230  
20        1.015369  
21        1.005167  
22        1.021864  
23        1.010455  
24        1.046789  
25        0.989992  
26        0.932230  
27        1.015369  
28        1.005167  
29        1.021864  
30        0.530890

In [117]:
for index, row in day_pred.iterrows(): # iterrows()可以把DataFrame按行读取
    if row['day'] == 31:
        break
    # 添加日期字段 report_date
    day_pred.loc[index,'report_date'] = pd.to_datetime('2014/09/'+str(int(row['day'])))
    
day_pred

day  total_purchase_amt  total_redeem_amt  purchase_weekday  \
0     1        3.222038e+08      2.335608e+08          1.009731   
1     2        2.464879e+08      2.139300e+08          1.006644   
2     3        2.887923e+08      2.937537e+08          1.037011   
3     4        3.065345e+08      2.794832e+08          1.030815   
4     5        3.277176e+08      2.578287e+08          1.002776   
5     6        3.204986e+08      2.639860e+08          0.935559   
6     7        2.837511e+08      2.338892e+08          0.999723   
7     8        2.890537e+08      2.462531e+08          1.009731   
8     9        2.837750e+08      2.388057e+08          1.006644   
9    10        3.323346e+08      2.567017e+08          1.037011   
10   11        2.778770e+08      2.915018e+08          1.030815   
11   12        2.739499e+08      2.574168e+08          1.002776   
12   13        2.650629e+08      2.939297e+08          0.935559   
13   14        2.709584e+08      2.906000e+08          0.999723   
14   15        2.929654e+08      2.290269e+08          1.009731   
15   16        3.123351e+08      3.058029e+08          1.006644   
16   17        2.473655e+08      2.956135e+08          1.037011   
17   18        2.719404e+08      2.673883e+08          1.030815   
18   19        2.829065e+08      2.634391e+08          1.002776   
19   20        3.111575e+08      2.539918e+08          0.935559   
20   21        2.816010e+08      2.775138e+08          0.999723   
21   22        2.406857e+08      2.599297e+08          1.009731   
22   23        2.267902e+08      2.569529e+08          1.006644   
23   24        2.321259e+08      2.942481e+08          1.037011   
24   25        2.357512e+08      3.015674e+08          1.030815   
25   26        2.496891e+08      3.148104e+08          1.002776   
26   27        2.547501e+08      3.409693e+08          0.935559   
27   28        2.663588e+08      3.408189e+08          0.999723   
28   29        2.419552e+08      2.677531e+08          1.009731   
29   30        2.377294e+08      2.874474e+08          1.006644   
30   31        4.729902e+08      5.353523e+08          0.535174   

    redeem_weekday report_date  
0         1.005167  2014-09-01  
1         1.021864  2014-09-02  
2         1.010455  2014-09-03  
3         1.046789  2014-09-04  
4         0.989992  2014-09-05  
5         0.932230  2014-09-06  
6         1.015369  2014-09-07  
7         1.005167  2014-09-08  
8         1.021864  2014-09-09  
9         1.010455  2014-09-10  
10        1.046789  2014-09-11  
11        0.989992  2014-09-12  
12        0.932230  2014-09-13  
13        1.015369  2014-09-14  
14        1.005167  2014-09-15  
15        1.021864  2014-09-16  
16        1.010455  2014-09-17  
17        1.046789  2014-09-18  
18        0.989992  2014-09-19  
19        0.932230  2014-09-20  
20        1.015369  2014-09-21  
21        1.005167  2014-09-22  
22        1.021864  2014-09-23  
23        1.010455  2014-09-24  
24        1.046789  2014-09-25  
25        0.989992  2014-09-26  
26        0.932230  2014-09-27  
27        1.015369  2014-09-28  
28        1.005167  2014-09-29  
29        1.021864  2014-09-30  
30        0.530890         NaT

In [118]:
# 利用周期因子 进行base * 周期因子 = 预测结果
day_pred['weekday'] = day_pred['report_date'].dt.weekday
day_pred['weekday'] = day_pred['weekday'][0:30].apply(lambda x:int(x)) # 转换为int型还是有小数点不知是为什么
day_pred

day  total_purchase_amt  total_redeem_amt  purchase_weekday  \
0     1        3.222038e+08      2.335608e+08          1.009731   
1     2        2.464879e+08      2.139300e+08          1.006644   
2     3        2.887923e+08      2.937537e+08          1.037011   
3     4        3.065345e+08      2.794832e+08          1.030815   
4     5        3.277176e+08      2.578287e+08          1.002776   
5     6        3.204986e+08      2.639860e+08          0.935559   
6     7        2.837511e+08      2.338892e+08          0.999723   
7     8        2.890537e+08      2.462531e+08          1.009731   
8     9        2.837750e+08      2.388057e+08          1.006644   
9    10        3.323346e+08      2.567017e+08          1.037011   
10   11        2.778770e+08      2.915018e+08          1.030815   
11   12        2.739499e+08      2.574168e+08          1.002776   
12   13        2.650629e+08      2.939297e+08          0.935559   
13   14        2.709584e+08      2.906000e+08          0.999723   
14   15        2.929654e+08      2.290269e+08          1.009731   
15   16        3.123351e+08      3.058029e+08          1.006644   
16   17        2.473655e+08      2.956135e+08          1.037011   
17   18        2.719404e+08      2.673883e+08          1.030815   
18   19        2.829065e+08      2.634391e+08          1.002776   
19   20        3.111575e+08      2.539918e+08          0.935559   
20   21        2.816010e+08      2.775138e+08          0.999723   
21   22        2.406857e+08      2.599297e+08          1.009731   
22   23        2.267902e+08      2.569529e+08          1.006644   
23   24        2.321259e+08      2.942481e+08          1.037011   
24   25        2.357512e+08      3.015674e+08          1.030815   
25   26        2.496891e+08      3.148104e+08          1.002776   
26   27        2.547501e+08      3.409693e+08          0.935559   
27   28        2.663588e+08      3.408189e+08          0.999723   
28   29        2.419552e+08      2.677531e+08          1.009731   
29   30        2.377294e+08      2.874474e+08          1.006644   
30   31        4.729902e+08      5.353523e+08          0.535174   

    redeem_weekday report_date  weekday  
0         1.005167  2014-09-01      0.0  
1         1.021864  2014-09-02      1.0  
2         1.010455  2014-09-03      2.0  
3         1.046789  2014-09-04      3.0  
4         0.989992  2014-09-05      4.0  
5         0.932230  2014-09-06      5.0  
6         1.015369  2014-09-07      6.0  
7         1.005167  2014-09-08      0.0  
8         1.021864  2014-09-09      1.0  
9         1.010455  2014-09-10      2.0  
10        1.046789  2014-09-11      3.0  
11        0.989992  2014-09-12      4.0  
12        0.932230  2014-09-13      5.0  
13        1.015369  2014-09-14      6.0  
14        1.005167  2014-09-15      0.0  
15        1.021864  2014-09-16      1.0  
16        1.010455  2014-09-17      2.0  
17        1.046789  2014-09-18      3.0  
18        0.989992  2014-09-19      4.0  
19        0.932230  2014-09-20      5.0  
20        1.015369  2014-09-21      6.0  
21        1.005167  2014-09-22      0.0  
22        1.021864  2014-09-23      1.0  
23        1.010455  2014-09-24      2.0  
24        1.046789  2014-09-25      3.0  
25        0.989992  2014-09-26      4.0  
26        0.932230  2014-09-27      5.0  
27        1.015369  2014-09-28      6.0  
28        1.005167  2014-09-29      0.0  
29        1.021864  2014-09-30      1.0  
30        0.530890         NaT      NaN

In [119]:
day_pred = day_pred[['day','total_purchase_amt','total_redeem_amt','report_date','weekday']]
day_pred

day  total_purchase_amt  total_redeem_amt report_date  weekday
0     1        3.222038e+08      2.335608e+08  2014-09-01      0.0
1     2        2.464879e+08      2.139300e+08  2014-09-02      1.0
2     3        2.887923e+08      2.937537e+08  2014-09-03      2.0
3     4        3.065345e+08      2.794832e+08  2014-09-04      3.0
4     5        3.277176e+08      2.578287e+08  2014-09-05      4.0
5     6        3.204986e+08      2.639860e+08  2014-09-06      5.0
6     7        2.837511e+08      2.338892e+08  2014-09-07      6.0
7     8        2.890537e+08      2.462531e+08  2014-09-08      0.0
8     9        2.837750e+08      2.388057e+08  2014-09-09      1.0
9    10        3.323346e+08      2.567017e+08  2014-09-10      2.0
10   11        2.778770e+08      2.915018e+08  2014-09-11      3.0
11   12        2.739499e+08      2.574168e+08  2014-09-12      4.0
12   13        2.650629e+08      2.939297e+08  2014-09-13      5.0
13   14        2.709584e+08      2.906000e+08  2014-09-14      6.0
14   15        2.929654e+08      2.290269e+08  2014-09-15      0.0
15   16        3.123351e+08      3.058029e+08  2014-09-16      1.0
16   17        2.473655e+08      2.956135e+08  2014-09-17      2.0
17   18        2.719404e+08      2.673883e+08  2014-09-18      3.0
18   19        2.829065e+08      2.634391e+08  2014-09-19      4.0
19   20        3.111575e+08      2.539918e+08  2014-09-20      5.0
20   21        2.816010e+08      2.775138e+08  2014-09-21      6.0
21   22        2.406857e+08      2.599297e+08  2014-09-22      0.0
22   23        2.267902e+08      2.569529e+08  2014-09-23      1.0
23   24        2.321259e+08      2.942481e+08  2014-09-24      2.0
24   25        2.357512e+08      3.015674e+08  2014-09-25      3.0
25   26        2.496891e+08      3.148104e+08  2014-09-26      4.0
26   27        2.547501e+08      3.409693e+08  2014-09-27      5.0
27   28        2.663588e+08      3.408189e+08  2014-09-28      6.0
28   29        2.419552e+08      2.677531e+08  2014-09-29      0.0
29   30        2.377294e+08      2.874474e+08  2014-09-30      1.0
30   31        4.729902e+08      5.353523e+08         NaT      NaN

In [120]:
day_pred = pd.merge(day_pred,weekday_weight,on='weekday')
day_pred

day  total_purchase_amt  total_redeem_amt report_date  weekday  \
0     1        3.222038e+08      2.335608e+08  2014-09-01      0.0   
1     8        2.890537e+08      2.462531e+08  2014-09-08      0.0   
2    15        2.929654e+08      2.290269e+08  2014-09-15      0.0   
3    22        2.406857e+08      2.599297e+08  2014-09-22      0.0   
4    29        2.419552e+08      2.677531e+08  2014-09-29      0.0   
5     2        2.464879e+08      2.139300e+08  2014-09-02      1.0   
6     9        2.837750e+08      2.388057e+08  2014-09-09      1.0   
7    16        3.123351e+08      3.058029e+08  2014-09-16      1.0   
8    23        2.267902e+08      2.569529e+08  2014-09-23      1.0   
9    30        2.377294e+08      2.874474e+08  2014-09-30      1.0   
10    3        2.887923e+08      2.937537e+08  2014-09-03      2.0   
11   10        3.323346e+08      2.567017e+08  2014-09-10      2.0   
12   17        2.473655e+08      2.956135e+08  2014-09-17      2.0   
13   24        2.321259e+08      2.942481e+08  2014-09-24      2.0   
14    4        3.065345e+08      2.794832e+08  2014-09-04      3.0   
15   11        2.778770e+08      2.915018e+08  2014-09-11      3.0   
16   18        2.719404e+08      2.673883e+08  2014-09-18      3.0   
17   25        2.357512e+08      3.015674e+08  2014-09-25      3.0   
18    5        3.277176e+08      2.578287e+08  2014-09-05      4.0   
19   12        2.739499e+08      2.574168e+08  2014-09-12      4.0   
20   19        2.829065e+08      2.634391e+08  2014-09-19      4.0   
21   26        2.496891e+08      3.148104e+08  2014-09-26      4.0   
22    6        3.204986e+08      2.639860e+08  2014-09-06      5.0   
23   13        2.650629e+08      2.939297e+08  2014-09-13      5.0   
24   20        3.111575e+08      2.539918e+08  2014-09-20      5.0   
25   27        2.547501e+08      3.409693e+08  2014-09-27      5.0   
26    7        2.837511e+08      2.338892e+08  2014-09-07      6.0   
27   14        2.709584e+08      2.906000e+08  2014-09-14      6.0   
28   21        2.816010e+08      2.775138e+08  2014-09-21      6.0   
29   28        2.663588e+08      3.408189e+08  2014-09-28      6.0   

    purchase_weekday  redeem_weekday  
0           1.185411        1.257964  
1           1.185411        1.257964  
2           1.185411        1.257964  
3           1.185411        1.257964  
4           1.185411        1.257964  
5           1.209608        1.172248  
6           1.209608        1.172248  
7           1.209608        1.172248  
8           1.209608        1.172248  
9           1.209608        1.172248  
10          1.163752        1.157944  
11          1.163752        1.157944  
12          1.163752        1.157944  
13          1.163752        1.157944  
14          1.139503        1.026762  
15          1.139503        1.026762  
16          1.139503        1.026762  
17          1.139503        1.026762  
18          0.902682        0.975445  
19          0.902682        0.975445  
20          0.902682        0.975445  
21          0.902682        0.975445  
22          0.709237        0.669791  
23          0.709237        0.669791  
24          0.709237        0.669791  
25          0.709237        0.669791  
26          0.712065        0.761712  
27          0.712065        0.761712  
28          0.712065        0.761712  
29          0.712065        0.761712

In [121]:
day_pred['total_purchase_amt'] *= day_pred['purchase_weekday']
day_pred['total_redeem_amt'] *= day_pred['redeem_weekday']
day_pred

day  total_purchase_amt  total_redeem_amt report_date  weekday  \
0     1        3.819439e+08      2.938111e+08  2014-09-01      0.0   
1     8        3.426474e+08      3.097775e+08  2014-09-08      0.0   
2    15        3.472844e+08      2.881076e+08  2014-09-15      0.0   
3    22        2.853115e+08      3.269821e+08  2014-09-22      0.0   
4    29        2.868163e+08      3.368237e+08  2014-09-29      0.0   
5     2        2.981538e+08      2.507790e+08  2014-09-02      1.0   
6     9        3.432566e+08      2.799395e+08  2014-09-09      1.0   
7    16        3.778032e+08      3.584768e+08  2014-09-16      1.0   
8    23        2.743273e+08      3.012125e+08  2014-09-23      1.0   
9    30        2.875595e+08      3.369596e+08  2014-09-30      1.0   
10    3        3.360825e+08      3.401504e+08  2014-09-03      2.0   
11   10        3.867549e+08      2.972462e+08  2014-09-10      2.0   
12   17        2.878720e+08      3.423040e+08  2014-09-17      2.0   
13   24        2.701369e+08      3.407229e+08  2014-09-24      2.0   
14    4        3.492969e+08      2.869627e+08  2014-09-04      3.0   
15   11        3.166415e+08      2.993030e+08  2014-09-11      3.0   
16   18        3.098768e+08      2.745442e+08  2014-09-18      3.0   
17   25        2.686391e+08      3.096380e+08  2014-09-25      3.0   
18    5        2.958247e+08      2.514977e+08  2014-09-05      4.0   
19   12        2.472895e+08      2.510959e+08  2014-09-12      4.0   
20   19        2.553745e+08      2.569703e+08  2014-09-19      4.0   
21   26        2.253898e+08      3.070802e+08  2014-09-26      4.0   
22    6        2.273095e+08      1.768153e+08  2014-09-06      5.0   
23   13        1.879924e+08      1.968713e+08  2014-09-13      5.0   
24   20        2.206844e+08      1.701213e+08  2014-09-20      5.0   
25   27        1.806782e+08      2.283780e+08  2014-09-27      5.0   
26    7        2.020493e+08      1.781561e+08  2014-09-07      6.0   
27   14        1.929401e+08      2.213535e+08  2014-09-14      6.0   
28   21        2.005183e+08      2.113855e+08  2014-09-21      6.0   
29   28        1.896649e+08      2.596058e+08  2014-09-28      6.0   

    purchase_weekday  redeem_weekday  
0           1.185411        1.257964  
1           1.185411        1.257964  
2           1.185411        1.257964  
3           1.185411        1.257964  
4           1.185411        1.257964  
5           1.209608        1.172248  
6           1.209608        1.172248  
7           1.209608        1.172248  
8           1.209608        1.172248  
9           1.209608        1.172248  
10          1.163752        1.157944  
11          1.163752        1.157944  
12          1.163752        1.157944  
13          1.163752        1.157944  
14          1.139503        1.026762  
15          1.139503        1.026762  
16          1.139503        1.026762  
17          1.139503        1.026762  
18          0.902682        0.975445  
19          0.902682        0.975445  
20          0.902682        0.975445  
21          0.902682        0.975445  
22          0.709237        0.669791  
23          0.709237        0.669791  
24          0.709237        0.669791  
25          0.709237        0.669791  
26          0.712065        0.761712  
27          0.712065        0.761712  
28          0.712065        0.761712  
29          0.712065        0.761712

In [122]:
day_pred = day_pred.sort_values('report_date')[['report_date','total_purchase_amt','total_redeem_amt']]
day_pred

report_date  total_purchase_amt  total_redeem_amt
0   2014-09-01        3.819439e+08      2.938111e+08
5   2014-09-02        2.981538e+08      2.507790e+08
10  2014-09-03        3.360825e+08      3.401504e+08
14  2014-09-04        3.492969e+08      2.869627e+08
18  2014-09-05        2.958247e+08      2.514977e+08
22  2014-09-06        2.273095e+08      1.768153e+08
26  2014-09-07        2.020493e+08      1.781561e+08
1   2014-09-08        3.426474e+08      3.097775e+08
6   2014-09-09        3.432566e+08      2.799395e+08
11  2014-09-10        3.867549e+08      2.972462e+08
15  2014-09-11        3.166415e+08      2.993030e+08
19  2014-09-12        2.472895e+08      2.510959e+08
23  2014-09-13        1.879924e+08      1.968713e+08
27  2014-09-14        1.929401e+08      2.213535e+08
2   2014-09-15        3.472844e+08      2.881076e+08
7   2014-09-16        3.778032e+08      3.584768e+08
12  2014-09-17        2.878720e+08      3.423040e+08
16  2014-09-18        3.098768e+08      2.745442e+08
20  2014-09-19        2.553745e+08      2.569703e+08
24  2014-09-20        2.206844e+08      1.701213e+08
28  2014-09-21        2.005183e+08      2.113855e+08
3   2014-09-22        2.853115e+08      3.269821e+08
8   2014-09-23        2.743273e+08      3.012125e+08
13  2014-09-24        2.701369e+08      3.407229e+08
17  2014-09-25        2.686391e+08      3.096380e+08
21  2014-09-26        2.253898e+08      3.070802e+08
25  2014-09-27        1.806782e+08      2.283780e+08
29  2014-09-28        1.896649e+08      2.596058e+08
4   2014-09-29        2.868163e+08      3.368237e+08
9   2014-09-30        2.875595e+08      3.369596e+08

In [123]:
day_pred_ri = day_pred_ri[['report_date','total_purchase_amt','total_redeem_amt']]
day_pred_ri

report_date  total_purchase_amt  total_redeem_amt
0     20140901        3.819439e+08      2.938111e+08
1     20140902        2.981538e+08      2.507790e+08
2     20140903        3.360825e+08      3.401504e+08
3     20140904        3.492969e+08      2.869627e+08
4     20140905        2.958247e+08      2.514977e+08
5     20140906        2.273095e+08      1.768153e+08
6     20140907        2.020493e+08      1.781561e+08
7     20140908        3.426474e+08      3.097775e+08
8     20140909        3.432566e+08      2.799395e+08
9     20140910        3.867549e+08      2.972462e+08
10    20140911        3.166415e+08      2.993030e+08
11    20140912        2.472895e+08      2.510959e+08
12    20140913        1.879924e+08      1.968713e+08
13    20140914        1.929401e+08      2.213535e+08
14    20140915        3.472844e+08      2.881076e+08
15    20140916        3.778032e+08      3.584768e+08
16    20140917        2.878720e+08      3.423040e+08
17    20140918        3.098768e+08      2.745442e+08
18    20140919        2.553745e+08      2.569703e+08
19    20140920        2.206844e+08      1.701213e+08
20    20140921        2.005183e+08      2.113855e+08
21    20140922        2.853115e+08      3.269821e+08
22    20140923        2.743273e+08      3.012125e+08
23    20140924        2.701369e+08      3.407229e+08
24    20140925        2.686391e+08      3.096380e+08
25    20140926        2.253898e+08      3.070802e+08
26    20140927        1.806782e+08      2.283780e+08
27    20140928        1.896649e+08      2.596058e+08
28    20140929        2.868163e+08      3.368237e+08
29    20140930        2.875595e+08      3.369596e+08

In [124]:
day_pred_ri['report_date'] = day_pred_ri['report_date'].apply(lambda x:str(x).replace('-','')[0:8])
day_pred_ri

report_date  total_purchase_amt  total_redeem_amt
0     20140901        3.819439e+08      2.938111e+08
1     20140902        2.981538e+08      2.507790e+08
2     20140903        3.360825e+08      3.401504e+08
3     20140904        3.492969e+08      2.869627e+08
4     20140905        2.958247e+08      2.514977e+08
5     20140906        2.273095e+08      1.768153e+08
6     20140907        2.020493e+08      1.781561e+08
7     20140908        3.426474e+08      3.097775e+08
8     20140909        3.432566e+08      2.799395e+08
9     20140910        3.867549e+08      2.972462e+08
10    20140911        3.166415e+08      2.993030e+08
11    20140912        2.472895e+08      2.510959e+08
12    20140913        1.879924e+08      1.968713e+08
13    20140914        1.929401e+08      2.213535e+08
14    20140915        3.472844e+08      2.881076e+08
15    20140916        3.778032e+08      3.584768e+08
16    20140917        2.878720e+08      3.423040e+08
17    20140918        3.098768e+08      2.745442e+08
18    20140919        2.553745e+08      2.569703e+08
19    20140920        2.206844e+08      1.701213e+08
20    20140921        2.005183e+08      2.113855e+08
21    20140922        2.853115e+08      3.269821e+08
22    20140923        2.743273e+08      3.012125e+08
23    20140924        2.701369e+08      3.407229e+08
24    20140925        2.686391e+08      3.096380e+08
25    20140926        2.253898e+08      3.070802e+08
26    20140927        1.806782e+08      2.283780e+08
27    20140928        1.896649e+08      2.596058e+08
28    20140929        2.868163e+08      3.368237e+08
29    20140930        2.875595e+08      3.369596e+08

In [125]:
day_pred_ri.to_csv('rule_base.csv',header=None,index=False)

finalScore:130.6812

将ARIMA和Rule_Base结果进行融合

In [132]:
data_arima = pd.read_csv('./arima.csv',header=None)
data_arima.columns

Int64Index([0, 1, 2], dtype='int64')

In [135]:
data_arima = data_arima.rename({0:'report_date',1:'total_purchase_amt',2:'total_redeem_amt'},axis=1)
data_arima

report_date  total_purchase_amt  total_redeem_amt
0      20140901        3.181209e+08      2.849093e+08
1      20140902        3.281642e+08      3.107281e+08
2      20140903        3.089999e+08      3.252729e+08
3      20140904        2.572063e+08      2.970501e+08
4      20140905        2.284246e+08      2.327450e+08
5      20140906        2.045706e+08      2.048514e+08
6      20140907        2.419994e+08      2.363191e+08
7      20140908        2.836054e+08      2.883918e+08
8      20140909        3.136331e+08      3.244628e+08
9      20140910        3.005516e+08      3.308337e+08
10     20140911        2.639080e+08      2.979443e+08
11     20140912        2.312889e+08      2.489066e+08
12     20140913        2.245305e+08      2.210997e+08
13     20140914        2.548090e+08      2.429546e+08
14     20140915        2.961682e+08      2.924717e+08
15     20140916        3.220720e+08      3.348743e+08
16     20140917        3.101161e+08      3.371792e+08
17     20140918        2.726888e+08      3.023099e+08
18     20140919        2.365145e+08      2.518969e+08
19     20140920        2.287316e+08      2.274582e+08
20     20140921        2.547156e+08      2.452222e+08
21     20140922        2.947057e+08      2.957865e+08
22     20140923        3.196762e+08      3.375856e+08
23     20140924        3.110034e+08      3.433539e+08
24     20140925        2.766563e+08      3.061767e+08
25     20140926        2.426236e+08      2.578075e+08
26     20140927        2.349973e+08      2.313834e+08
27     20140928        2.594247e+08      2.508561e+08
28     20140929        2.979944e+08      2.987885e+08
29     20140930        3.224251e+08      3.430741e+08

In [136]:
data_fuse = pd.DataFrame()
data_fuse['report_date'] = data_arima['report_date']
data_fuse['total_purchase_amt'] = (day_pred_ri['total_purchase_amt']+data_arima['total_purchase_amt'])/2
data_fuse['total_redeem_amt'] = (day_pred_ri['total_redeem_amt']+data_arima['total_redeem_amt'])/2

In [137]:
data_fuse

report_date  total_purchase_amt  total_redeem_amt
0      20140901        3.500324e+08      2.893602e+08
1      20140902        3.131590e+08      2.807535e+08
2      20140903        3.225412e+08      3.327116e+08
3      20140904        3.032516e+08      2.920064e+08
4      20140905        2.621246e+08      2.421213e+08
5      20140906        2.159400e+08      1.908334e+08
6      20140907        2.220244e+08      2.072376e+08
7      20140908        3.131264e+08      2.990847e+08
8      20140909        3.284448e+08      3.022011e+08
9      20140910        3.436533e+08      3.140400e+08
10     20140911        2.902748e+08      2.986237e+08
11     20140912        2.392892e+08      2.500013e+08
12     20140913        2.062615e+08      2.089855e+08
13     20140914        2.238746e+08      2.321540e+08
14     20140915        3.217263e+08      2.902896e+08
15     20140916        3.499376e+08      3.466756e+08
16     20140917        2.989940e+08      3.397416e+08
17     20140918        2.912828e+08      2.884271e+08
18     20140919        2.459445e+08      2.544336e+08
19     20140920        2.247080e+08      1.987898e+08
20     20140921        2.276169e+08      2.283039e+08
21     20140922        2.900086e+08      3.113843e+08
22     20140923        2.970018e+08      3.193990e+08
23     20140924        2.905702e+08      3.420384e+08
24     20140925        2.726477e+08      3.079073e+08
25     20140926        2.340067e+08      2.824439e+08
26     20140927        2.078377e+08      2.298807e+08
27     20140928        2.245448e+08      2.552310e+08
28     20140929        2.924053e+08      3.178061e+08
29     20140930        3.049923e+08      3.400168e+08

In [138]:
data_fuse.to_csv('arima_rulebase_fuse.csv',header=None,index=False)

finalScore:117.9004 融合后的分数并没有提高